## Метод 1

В даному методі використовується байєсівський метод оптимізації, де ми мінімізуємо штрафну функцію та зберігаємо завжди найменше значення функції та зберігаємо q, при якому це значення вийшло.

In [1]:
import numpy as np

def penalty_function(q, sequence1, sequence2):
    # Розрахунок штрафної функції з параметром q
    penalty = (sum(sequence1) - sum(sequence2))**2 * (q - sum(sequence2))**2
    return penalty

def generate_sequence(length):
    # Генеруємо випадкову послідовність з 0 та 1
    initial_sequence = np.random.choice([0, 1], size=length)

    # Змінюємо кожен елемент з ймовірністю 50%
    modified_sequence = [np.random.choice([0, 1], p=[0.5, 0.5]) for _ in range(length)]

    return initial_sequence, modified_sequence

# Приклад використання
sequence_length = 10
initial_sequence, modified_sequence = generate_sequence(sequence_length)

# Користувацький байєсівський метод
best_q = None
best_penalty = float('inf')

for q_candidate in range(11):
    current_penalty = penalty_function(q_candidate, initial_sequence, modified_sequence)
    
    if current_penalty < best_penalty:
        best_penalty = current_penalty
        best_q = q_candidate

# Вивід результатів
print(f"Початкова послідовність: {initial_sequence}")
print(f"Модифікована послідовність: {modified_sequence}")
print(f"Штрафна функція для оптимального q: {best_penalty}")
print(f"Оптимальне значення q: {best_q}")


Початкова послідовність: [0 0 1 0 0 0 1 1 0 1]
Модифікована послідовність: [0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
Штрафна функція для оптимального q: 0
Оптимальне значення q: 5


## Метод 2

Основа суть даного числового методу полягає в тому, щоб на кожному кроці оптимізації новий кандидат для оптимальних параметрів знаходився в певних визначених межах. У контексті байєсівської оптимізації, L-BFGS-B використовується як метод локальної оптимізації на кожному кроці ітеративного байєсівського оптимізатора для знаходження найкращих параметрів.




In [2]:
import numpy as np
from scipy.optimize import minimize

def penalty_function(q, sequence1, sequence2):
    # Розрахунок штрафної функції з параметром q
    penalty = sum((x - y)**2 * (q - y)**2 for x, y in zip(sequence1, sequence2))
    return penalty

def generate_sequence(length):
    # Генеруємо випадкову послідовність з 0 та 1
    initial_sequence = np.random.choice([0, 1], size=length)

    # Змінюємо кожен елемент з ймовірністю 50%
    modified_sequence = [np.random.choice([0, 1], p=[0.5, 0.5]) for _ in range(length)]

    return initial_sequence, modified_sequence

# Приклад використання
sequence_length = 10
initial_sequence, modified_sequence = generate_sequence(sequence_length)

# Задаємо початкове значення для q
initial_q = 0.0

# Мінімізуємо штрафну функцію з параметром q
result = minimize(penalty_function, initial_q, args=(initial_sequence, modified_sequence), method='L-BFGS-B')

# Отримуємо оптимальне значення q
optimal_q = result.x[0]

# Розраховуємо штрафну функцію для отриманого q
optimal_penalty = penalty_function(optimal_q, initial_sequence, modified_sequence)

print(f"Початкова послідовність: {initial_sequence}")
print(f"Модифікована послідовність: {modified_sequence}")
print(f"Штрафна функція для оптимального q: {optimal_penalty}")
print(f"Оптимальне значення q: {optimal_q}")


Початкова послідовність: [0 1 1 0 0 0 1 0 0 0]
Модифікована послідовність: [1, 1, 0, 1, 1, 1, 0, 1, 0, 0]
Штрафна функція для оптимального q: 1.4285714285714284
Оптимальне значення q: 0.7142857130454141


## Метод 3

Тут використовується метод оптимізації Sequential Model-Based Optimization (SMBO), який реалізований через функцію gp_minimize з бібліотеки scikit-optimize (skopt). Основна ідея SMBO полягає в тому, щоб використовувати гауссівські процеси (Gaussian Processes, GP) для створення ймовірнісних моделей функцій, які оцінюють, наскільки гарно працює кожен варіант параметрів.

In [3]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt import space


def penalty_function(q, sequence1, sequence2):
    # Розрахунок штрафної функції з параметром q
    penalty = (sum(sequence1) - sum(sequence2))**2 * (q - sum(sequence2))**2
    return penalty


def generate_sequence(length):
    # Генеруємо випадкову послідовність з 0 та 1
    initial_sequence = np.random.choice([0, 1], size=length)

    # Змінюємо кожен елемент з ймовірністю 50%
    modified_sequence = [np.random.choice([0, 1], p=[0.5, 0.5]) for _ in range(length)]

    return initial_sequence, modified_sequence

# Приклад використання
sequence_length = 10
initial_sequence, modified_sequence = generate_sequence(sequence_length)

# Задаємо простір пошуку для q
space = [Real(0.0, 10.0, name='q')]  # Replace 1 and 10 with your desired range for q

# Оптимізуємо штрафну функцію з параметром q за допомогою SMBO
result = gp_minimize(
    lambda q: penalty_function(q[0], initial_sequence, modified_sequence),
    space,
    n_calls=10,
    random_state=42,
    verbose=1
)

# Отримуємо оптимальне значення q
optimal_q = result.x[0]

# Розраховуємо штрафну функцію для отриманого q
optimal_penalty = penalty_function(optimal_q, initial_sequence, modified_sequence)

print(f"Початкова послідовність: {initial_sequence}")
print(f"Модифікована послідовність: {modified_sequence}")
print(f"Штрафна функція для оптимального q: {optimal_penalty}")
print(f"Оптимальне значення q: {round(optimal_q)}")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0010
Function value obtained: 3.8629
Current minimum: 3.8629
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 17.3527
Current minimum: 3.8629
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 3.2289
Current minimum: 3.2289
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 0.0010
Current minimum: 0.0010
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0000
Function value obtained: 2.3768
Current minimum: 0.0010
Iteration No: 6 started. Evaluatin